# Airbnb property price

### Loading the dataset

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("PromptCloudHQ/airbnb-property-data-from-texas")

print("Path to dataset files:", path)

100%|██████████| 3.19M/3.19M [00:57<00:00, 58.0kB/s]

Extracting files...
Path to dataset files: C:\Users\Asus\.cache\kagglehub\datasets\PromptCloudHQ\airbnb-property-data-from-texas\versions\1


### Importing libraries

In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns